In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

# Calculations
import math
from haversine import haversine, Unit
from geopy.distance import geodesic
from shapely.geometry import Point

# ML libraries
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from gluonts.torch.model.tft import TemporalFusionTransformerEstimator  # TFT from Torch
from gluonts.torch.model.deepar import DeepAREstimator  # DeepAR from Torch
from lightning.pytorch import Trainer  # PyTorch Lightning Trainer


from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
import torch
import torch.nn as nn
import torch.optim as optim  # For optimization

# AutoML libraries
from autogluon.tabular import TabularPredictor


### Models Employed
\
tabularPredictor
\
GRU-D 
\
Both of these for one-step recurrent prediction
\
TFT
\
deepAR
\
Both of these for sequence prediction
\
These will be stacked using autogluon


In [2]:
# Define the path to your file in the bucket
file_path = 'gs://jacobsbucketformlproject/ML Competition/ais_train.csv'

# Load the file into a pandas dataframe
ais_train_df = pd.read_csv(file_path, delimiter= '|', encoding= 'utf-8')

# Display the dataframe
ais_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522065 entries, 0 to 1522064
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   time       1522065 non-null  object 
 1   cog        1522065 non-null  float64
 2   sog        1522065 non-null  float64
 3   rot        1522065 non-null  int64  
 4   heading    1522065 non-null  int64  
 5   navstat    1522065 non-null  int64  
 6   etaRaw     1522065 non-null  object 
 7   latitude   1522065 non-null  float64
 8   longitude  1522065 non-null  float64
 9   vesselId   1522065 non-null  object 
 10  portId     1520450 non-null  object 
dtypes: float64(4), int64(3), object(4)
memory usage: 127.7+ MB


In [3]:
ais_test_df = pd.read_csv('gs://jacobsbucketformlproject/ML Competition/ais_test.csv')
ais_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51739 entries, 0 to 51738
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              51739 non-null  int64  
 1   vesselId        51739 non-null  object 
 2   time            51739 non-null  object 
 3   scaling_factor  51739 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.6+ MB


In [4]:
# Define the path to your file in the bucket
file_path = 'gs://jacobsbucketformlproject/ML Competition/ports.csv'

#Load the file into a pandas dataframe
ports_df = pd.read_csv(file_path, delimiter= '|', encoding= 'utf-8')

ports_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329 entries, 0 to 1328
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   portId        1329 non-null   object 
 1   name          1329 non-null   object 
 2   portLocation  1329 non-null   object 
 3   longitude     1329 non-null   float64
 4   latitude      1329 non-null   float64
 5   UN_LOCODE     1329 non-null   object 
 6   countryName   1329 non-null   object 
 7   ISO           1329 non-null   object 
dtypes: float64(2), object(6)
memory usage: 83.2+ KB


In [5]:
# Define the path to your file in the bucket
file_path = 'gs://jacobsbucketformlproject/ML Competition/vessels.csv'

#Load the file into a pandas dataframe
vessels_df = pd.read_csv(file_path, delimiter= '|', encoding= 'utf-8')

vessels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   shippingLineId  711 non-null    object 
 1   vesselId        711 non-null    object 
 2   CEU             711 non-null    int64  
 3   DWT             703 non-null    float64
 4   GT              711 non-null    int64  
 5   NT              187 non-null    float64
 6   vesselType      699 non-null    float64
 7   breadth         703 non-null    float64
 8   depth           242 non-null    float64
 9   draft           10 non-null     float64
 10  enginePower     691 non-null    float64
 11  freshWater      221 non-null    float64
 12  fuel            221 non-null    float64
 13  homePort        573 non-null    object 
 14  length          711 non-null    float64
 15  maxHeight       35 non-null     float64
 16  maxSpeed        213 non-null    float64
 17  maxWidth        35 non-null     flo

In [6]:
# Define the path to your file in the bucket
file_path = 'gs://jacobsbucketformlproject/ML Competition/schedules_to_may_2024.csv'

#Load the file into a pandas dataframe
schedules_df = pd.read_csv(file_path, delimiter= '|', encoding= 'utf-8')

# We pre-process ais_train and combine it with the relevant features from ports:

In [7]:
def preprocess_ais_train(ais_train_df, ports_df):
    """
    Preprocess the ais_train_df by converting columns, handling missing or invalid values, 
    merging port information, and mapping NAVSTAT codes to descriptions.
    
    Additionally, set 'etaRaw' to NaN if its value is less than the current time.

    Parameters:
    - ais_train_df: DataFrame containing the raw AIS train data.
    - ports_df: DataFrame containing port information with portId, latitude, and longitude.

    Returns:
    - ais_train_df_cleaned: A cleaned and preprocessed version of ais_train_df.
    """
    # Step 1: Convert 'time' and 'etaRaw' to datetime
    ais_train_df['time'] = pd.to_datetime(ais_train_df['time'], format='%Y-%m-%d %H:%M:%S')
    ais_train_df['etaRaw'] = pd.to_datetime(ais_train_df['etaRaw'], errors='coerce', format='%m-%d %H:%M')
    
    # Step 2: Add the year 2024 to 'etaRaw' column
    ais_train_df['etaRaw'] = ais_train_df['etaRaw'].apply(lambda x: x.replace(year=2024) if pd.notna(x) else pd.NaT)

    # Step 3: Set etaRaw to NaN if etaRaw is less than the current time
    current_time = pd.Timestamp.now()
    ais_train_df['etaRaw'] = ais_train_df.apply(lambda row: row['etaRaw'] if pd.isna(row['etaRaw']) or row['etaRaw'] >= row['time'] else pd.NaT, axis=1)

    # Step 4: Convert relevant columns to float
    ais_train_df['cog'] = ais_train_df['cog'].astype(float)
    ais_train_df['sog'] = ais_train_df['sog'].astype(float)
    ais_train_df['rot'] = ais_train_df['rot'].astype(float)
    ais_train_df['heading'] = ais_train_df['heading'].astype(float)
    ais_train_df['latitude'] = ais_train_df['latitude'].astype(float)
    ais_train_df['longitude'] = ais_train_df['longitude'].astype(float)
    
    # Step 5: Replace invalid or default values with NaN
    ais_train_df['cog'] = np.where((ais_train_df['cog'] == 360) | (ais_train_df['cog'] > 360) | (ais_train_df['cog'] < 0), np.nan, ais_train_df['cog'])
    ais_train_df['sog'] = np.where((ais_train_df['sog'] == 1023) | (ais_train_df['sog'] < 0), np.nan, ais_train_df['sog'])
    ais_train_df['rot'] = np.where((ais_train_df['rot'] == -128), np.nan, ais_train_df['rot'])
    ais_train_df['heading'] = np.where((ais_train_df['heading'] > 360) | (ais_train_df['heading'] == 511) | (ais_train_df['heading'] < 0), np.nan, ais_train_df['heading'])

    # Step 6: Merge with ports to get port latitude and longitude

    # Renaming the latitude and longitude columns in ports_df to portLatitude and portLongitude
    ports_df = ports_df.rename(columns={'latitude': 'portLatitude', 'longitude': 'portLongitude'})

    # Checking for missing portId values and printing for reference
    missing_in_ports = set(ais_train_df['portId']) - set(ports_df['portId'])
    if len(missing_in_ports) > 0:
        print(f"Warning: {len(missing_in_ports)} portId values from ais_train_df do not exist in ports_df.")
    
    # Merging ais_train_df with the updated ports_df on 'portId'
    ais_train_df = ais_train_df.merge(ports_df[['portId', 'portLatitude', 'portLongitude']], on='portId', how='left')
    
    # Step 7: Sort by vesselId and time
    ais_train_df = ais_train_df.sort_values(by=['vesselId', 'time']).reset_index(drop=True)

    return ais_train_df

ais_train_df = preprocess_ais_train(ais_train_df, ports_df)

ais_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522065 entries, 0 to 1522064
Data columns (total 13 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   time           1522065 non-null  datetime64[ns]
 1   cog            1516207 non-null  float64       
 2   sog            1522065 non-null  float64       
 3   rot            1522065 non-null  float64       
 4   heading        1517169 non-null  float64       
 5   navstat        1522065 non-null  int64         
 6   etaRaw         766829 non-null   datetime64[ns]
 7   latitude       1522065 non-null  float64       
 8   longitude      1522065 non-null  float64       
 9   vesselId       1522065 non-null  object        
 10  portId         1520450 non-null  object        
 11  portLatitude   1520450 non-null  float64       
 12  portLongitude  1520450 non-null  float64       
dtypes: datetime64[ns](2), float64(8), int64(1), object(2)
memory usage: 151.0+ MB


# We now pre-process the vessels

In [8]:
def preprocess_vessels(vessels_df):
    """
    Preprocess the vessels_df by converting 'yearBuilt' to 'age', handling missing values, 
    mapping 'homePort', and converting 'shippingLineId' into a categorical feature.
    
    Parameters:
    - vessels_df: DataFrame containing the raw vessels data.
    
    Returns:
    - vessels_df_cleaned: A cleaned and preprocessed version of vessels_df.
    """
    
    # Step 1: Convert 'yearBuilt' to 'age'
    current_year = 2024
    vessels_df['age'] = vessels_df['yearBuilt'].apply(lambda x: current_year - x if pd.notna(x) else np.nan)
    vessels_df.drop(columns=['yearBuilt'], inplace=True)
    
    # Step 2: Drop columns with high missing values and low predictive power
    columns_to_drop = ['NT', 'depth', 'draft', 'freshWater', 'enginePower', 'fuel', 
                       'maxHeight', 'maxWidth', 'rampCapacity', 'maxSpeed']
    vessels_df.drop(columns=columns_to_drop, inplace=True)
    
    # Step 3: Make vesselType into category
    # Convert 'vesselType' from float to categorical without knowing the exact mapping
    vessels_df['vesselType'] = vessels_df['vesselType'].astype('category')

    # Optionally, handle missing values (nan) by filling them with 'Unknown' or leaving them as is
    vessels_df['vesselType'] = vessels_df['vesselType'].cat.add_categories('Unknown').fillna('Unknown')
    
    return vessels_df


def map_homePort_to_country(vessels_df):
    """
    Maps 'homePort' city names to their respective countries and groups rare countries into 'OTHER'.
    """
    initial_mapping = {
        'PANAMA': 'Panama', 'UNKNOWN': 'Unknown', 'PALERMO': 'Italy', 'NASSAU': 'Bahamas', 
        'TOKYO': 'Japan', 'VALLETTA': 'Malta', 'OSLO': 'Norway', 'MONROVIA': 'Liberia', 
        'MAJURO': 'Marshall Islands', 'JEJU CHEJU': 'South Korea', 'HELSINKI': 'Finland',
        # Add other mappings here...
    }
    
    vessels_df['homePort'] = vessels_df['homePort'].map(initial_mapping).fillna('OTHER')
    
    # Group rare countries into 'OTHER' (with fewer than 10 occurrences)
    country_counts = vessels_df['homePort'].value_counts()
    rare_countries = country_counts[country_counts < 10].index.tolist()
    vessels_df['homePort'] = vessels_df['homePort'].replace(rare_countries, 'OTHER')
    
    return vessels_df


def process_shippingLineId(vessels_df):
    """
    Converts 'shippingLineId' into a categorical feature, grouping those with less than 13 occurrences 
    into 'Unknown'.
    """
    # Calculate the frequency of each shippingLineId
    shipping_freq = vessels_df['shippingLineId'].value_counts()
    
    # Map rare shippingLineIds (occurring less than 13 times) to 'Unknown'
    vessels_df['shippingLineId'] = vessels_df['shippingLineId'].apply(
        lambda x: x if shipping_freq[x] >= 13 else 'Unknown')
    
    return vessels_df


def knn_impute_data(vessels_df, numerical_features, predictor_features, n_neighbors=10):
    """
    Performs KNN imputation on the specified numerical features using the predictor features.
    """
    
    # Prepare DataFrame for imputation
    imputation_df = vessels_df[numerical_features + predictor_features].copy()
    
    # Ensure that only numerical columns are included for scaling and imputation
    imputation_df = imputation_df.apply(pd.to_numeric, errors='coerce')
    
    # Scale numerical features
    scaler = StandardScaler()
    imputation_df[numerical_features] = scaler.fit_transform(imputation_df[numerical_features])
    
    # Apply KNN Imputer
    knn_imputer = KNNImputer(n_neighbors=n_neighbors, weights="distance")
    imputed_values = knn_imputer.fit_transform(imputation_df)
    
    # Convert imputed values back to DataFrame and rescale
    imputed_df = pd.DataFrame(imputed_values, columns=numerical_features + predictor_features)
    imputed_df[numerical_features] = scaler.inverse_transform(imputed_df[numerical_features])
    
    # Assign imputed values back to the original vessel DataFrame
    for feature in numerical_features:
        vessels_df[feature] = imputed_df[feature]
    
    return vessels_df


# Main Preprocessing Pipeline
def preprocess_all(vessels_df):
    """
    Full preprocessing pipeline for vessels_df including handling shippingLineId, homePort,
    KNN imputation, and maxSpeed adjustments.
    """
    # Step 1: Basic preprocessing (convert yearBuilt to age and drop unnecessary columns)
    vessels_df = preprocess_vessels(vessels_df)
    
    # Step 2: Map 'homePort' to country
    vessels_df = map_homePort_to_country(vessels_df)
    
    # Step 3: Convert 'shippingLineId' to categorical with grouping of rare values
    vessels_df = process_shippingLineId(vessels_df)
    
    # Step 4: Impute missing numerical features
    numerical_features = ['DWT', 'breadth']
    predictor_features = ['CEU', 'GT', 'length', 'age']  # Exclude non-numeric features
    vessels_df = knn_impute_data(vessels_df, numerical_features, predictor_features)
    
    return vessels_df


# Apply the full pipeline
vessels_df = preprocess_all(vessels_df)
vessels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   shippingLineId  711 non-null    object  
 1   vesselId        711 non-null    object  
 2   CEU             711 non-null    int64   
 3   DWT             711 non-null    float64 
 4   GT              711 non-null    int64   
 5   vesselType      711 non-null    category
 6   breadth         711 non-null    float64 
 7   homePort        711 non-null    object  
 8   length          711 non-null    float64 
 9   age             711 non-null    int64   
dtypes: category(1), float64(3), int64(3), object(3)
memory usage: 51.0+ KB


In [9]:
# Check unique values to understand the categories
print(vessels_df['vesselType'].unique())

[83.0, 'Unknown', 21.0, 14.0]
Categories (4, object): [14.0, 21.0, 83.0, 'Unknown']


### Wait with schedules for now, as only 67 of schedules are for vessels in test set, and there are few schedules that match with ais, hence, a lot of Nan values. Perhaps use it for common ports or routes, but I feel like we could get this more realiably from the ais data

# We now merge vessel data with ais to get a base dataset

In [10]:
def merge_ais_with_vessels(ais_data, vessel_data):
    """
    Merges the AIS data with the vessel data on 'vesselId' using a left join.
    
    Parameters:
    - ais_data (pd.DataFrame): DataFrame containing AIS data.
    - vessel_data (pd.DataFrame): DataFrame containing vessel data.
    
    Returns:
    - baseDataset (pd.DataFrame): Merged DataFrame containing the AIS data with vessel information.
    """
    # Perform a left merge on 'vesselId'
    baseDataset = pd.merge(ais_data, vessel_data, how='left', on='vesselId')
    
    return baseDataset

# Example usage:
baseDataset = merge_ais_with_vessels(ais_train_df, vessels_df)


In [11]:
baseDataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522065 entries, 0 to 1522064
Data columns (total 22 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   time            1522065 non-null  datetime64[ns]
 1   cog             1516207 non-null  float64       
 2   sog             1522065 non-null  float64       
 3   rot             1522065 non-null  float64       
 4   heading         1517169 non-null  float64       
 5   navstat         1522065 non-null  int64         
 6   etaRaw          766829 non-null   datetime64[ns]
 7   latitude        1522065 non-null  float64       
 8   longitude       1522065 non-null  float64       
 9   vesselId        1522065 non-null  object        
 10  portId          1520450 non-null  object        
 11  portLatitude    1520450 non-null  float64       
 12  portLongitude   1520450 non-null  float64       
 13  shippingLineId  1522065 non-null  object        
 14  CEU             15

In [12]:
baseDataset.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,...,portLongitude,shippingLineId,CEU,DWT,GT,vesselType,breadth,homePort,length,age
0,2024-01-12 14:07:47,308.1,17.1,-6.0,316.0,0,NaT,7.50361,77.58340,61e9f38eb937134a3c4bfd8b,...,80.341111,61a8e672f9cba188601e84ab,6500,21200.0,58684,83.0,32.0,Norway,199.0,24
1,2024-01-12 14:31:00,307.6,17.3,5.0,313.0,0,2024-01-14 23:30:00,7.57302,77.49505,61e9f38eb937134a3c4bfd8b,...,72.885278,61a8e672f9cba188601e84ab,6500,21200.0,58684,83.0,32.0,Norway,199.0,24
2,2024-01-12 14:57:23,306.8,16.9,5.0,312.0,0,2024-01-14 23:30:00,7.65043,77.39404,61e9f38eb937134a3c4bfd8b,...,72.885278,61a8e672f9cba188601e84ab,6500,21200.0,58684,83.0,32.0,Norway,199.0,24
3,2024-01-12 15:18:48,307.9,16.9,6.0,313.0,0,2024-01-14 23:30:00,7.71275,77.31394,61e9f38eb937134a3c4bfd8b,...,72.885278,61a8e672f9cba188601e84ab,6500,21200.0,58684,83.0,32.0,Norway,199.0,24
4,2024-01-12 15:39:47,307.0,16.3,7.0,313.0,0,2024-01-14 23:30:00,7.77191,77.23585,61e9f38eb937134a3c4bfd8b,...,72.885278,61a8e672f9cba188601e84ab,6500,21200.0,58684,83.0,32.0,Norway,199.0,24


# We now remove anomalies and impute some missing values for this dataset before moving onto feature engineering
This includes removing vessels with very few records
Flagging unusually large gaps and getting time diff feature
Fixing sog values
Fixing cog values
Fixing heading values
Fixing rot values
generalizing navstat to fewer more meaningful values


We check to see the vessels with least records, and whether they are inn test set, and hence must be in dataset! If we find few records we remove them from the dataset

In [13]:
# Get the unique vesselIds from the test set
vessel_ids_test = set(ais_test_df['vesselId'].unique())

# Get the count of records per vesselId in the training set
vessel_record_counts = ais_train_df['vesselId'].value_counts()

# Get the 10 vessels with the lowest number of records
lowest_record_vessels = vessel_record_counts.nsmallest(20)

# Check if these vessels are in the test set
vessels_in_test = lowest_record_vessels.index.isin(vessel_ids_test)

# Combine the results into a dataframe for easy viewing
vessels_with_low_records = pd.DataFrame({
    'vesselId': lowest_record_vessels.index,
    'record_count': lowest_record_vessels.values,
    'in_test_set': vessels_in_test
})

# Display the result
print(vessels_with_low_records)

                    vesselId  record_count  in_test_set
0   61e9f3cbb937134a3c4bff09             1        False
1   61e9f3adb937134a3c4bfe37            31        False
2   61e9f3c6b937134a3c4bfed5           160        False
3   61e9f42cb937134a3c4c00f9           191        False
4   61e9f45cb937134a3c4c022b           196        False
5   61e9f39ab937134a3c4bfdb9           197        False
6   61e9f45eb937134a3c4c0235           250        False
7   61e9f3bcb937134a3c4bfe91           328         True
8   61e9f418b937134a3c4c0077           332        False
9   61e9f408b937134a3c4c0023           355        False
10  61e9f460b937134a3c4c0243           361        False
11  61e9f3f7b937134a3c4bffc5           373        False
12  61e9f409b937134a3c4c0027           391        False
13  620bf33a718775aca4a81900           401        False
14  61e9f423b937134a3c4c00c7           402        False
15  61e9f38eb937134a3c4bfd8b           402        False
16  61e9f456b937134a3c4c0203           408      

We remove the two vessels with the lowest records count

In [14]:
# List of vessel IDs to remove
vessels_to_remove = ['61e9f3cbb937134a3c4bff09', '61e9f3adb937134a3c4bfe37']

# Remove vessels from the dataset
baseDataset = baseDataset[~ais_train_df['vesselId'].isin(vessels_to_remove)]

We now create a time_diff column and flag large time gaps column

In [15]:
def add_time_diff_and_gap_flag(df, time_col='time', vessel_col='vesselId', threshold=2):
    """
    Adds time_diff and large_gap_flag to the dataframe based on vessel-specific statistics.
    
    Parameters:
    df (pd.DataFrame): The input dataframe containing vesselId and timestamp columns.
    time_col (str): The name of the timestamp column.
    vessel_col (str): The name of the vessel identifier column.
    threshold (float): The number of standard deviations above the mean to flag large time gaps.
    
    Returns:
    pd.DataFrame: The original dataframe with 'time_diff' and 'large_gap_flag' added.
    """
    # Ensure the time column is in datetime format
    df[time_col] = pd.to_datetime(df[time_col])
    
    # Calculate time_diff (difference in time between consecutive entries for each vessel)
    df['time_diff'] = df.groupby(vessel_col)[time_col].diff().dt.total_seconds().fillna(0)
    
    # Group by vesselId to calculate the mean and standard deviation of time_diff for each vessel
    vessel_stats = df.groupby(vessel_col)['time_diff'].agg(['mean', 'std']).reset_index()
    
    # Merge vessel statistics back to the original dataframe
    df = df.merge(vessel_stats, on=vessel_col, how='left')
    
    # Define a large time gap as threshold * standard deviations above the mean for each vessel
    df['large_gap_flag'] = (df['time_diff'] > df['mean'] + threshold * df['std']).astype(int)
    
    # Drop the mean and std columns (optional, you can keep them if needed)
    df = df.drop(columns=['mean', 'std'])
    
    return df


In [16]:
baseDataset = add_time_diff_and_gap_flag(baseDataset, time_col='time', vessel_col='vesselId', threshold=3)
baseDataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522033 entries, 0 to 1522032
Data columns (total 24 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   time            1522033 non-null  datetime64[ns]
 1   cog             1516175 non-null  float64       
 2   sog             1522033 non-null  float64       
 3   rot             1522033 non-null  float64       
 4   heading         1517137 non-null  float64       
 5   navstat         1522033 non-null  int64         
 6   etaRaw          766797 non-null   datetime64[ns]
 7   latitude        1522033 non-null  float64       
 8   longitude       1522033 non-null  float64       
 9   vesselId        1522033 non-null  object        
 10  portId          1520418 non-null  object        
 11  portLatitude    1520418 non-null  float64       
 12  portLongitude   1520418 non-null  float64       
 13  shippingLineId  1522033 non-null  object        
 14  CEU             15

# We now generalize the navstat for easier use

In [17]:
def map_navstat_to_movement(df, navstat_col='navstat'):
    """
    Create a new feature indicating whether the vessel is moving, anchored, or moored 
    based on the navstat column and then remove the original navstat column.

    Parameters:
    df (pd.DataFrame): The input dataframe containing the navstat column.
    navstat_col (str): The name of the navstat column.

    Returns:
    pd.DataFrame: The dataframe with a new 'vessel_status' column and without the original 'navstat' column.
    """
    # Define the mapping of navstat codes to the movement categories
    navstat_mapping = {
        0: 'moving',      # Underway using engine
        1: 'anchored',    # Anchored
        2: 'unknown',      # Not under command
        3: 'moving',      # Restricted manoeuverability
        4: 'moving',      # Constrained by her draught
        5: 'moored',      # Moored
        6: 'anchored',    # Aground (considered stationary)
        7: 'moving',      # Engaged in fishing
        8: 'moving',      # Underway sailing
        9: 'unknown',      # Reserved for future use
        15: 'unknown'      # Undefined
    }

    # Create a new column based on the mapping
    df['vessel_status'] = df[navstat_col].map(navstat_mapping)

    # Remove the original navstat column
    df = df.drop(columns=[navstat_col])

    return df

#Apply the function to create the vessel_status feature and remove the navstat column
baseDataset = map_navstat_to_movement(baseDataset, navstat_col='navstat')

In [18]:
baseDataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522033 entries, 0 to 1522032
Data columns (total 24 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   time            1522033 non-null  datetime64[ns]
 1   cog             1516175 non-null  float64       
 2   sog             1522033 non-null  float64       
 3   rot             1522033 non-null  float64       
 4   heading         1517137 non-null  float64       
 5   etaRaw          766797 non-null   datetime64[ns]
 6   latitude        1522033 non-null  float64       
 7   longitude       1522033 non-null  float64       
 8   vesselId        1522033 non-null  object        
 9   portId          1520418 non-null  object        
 10  portLatitude    1520418 non-null  float64       
 11  portLongitude   1520418 non-null  float64       
 12  shippingLineId  1522033 non-null  object        
 13  CEU             1522033 non-null  int64         
 14  DWT             15

In [19]:
def impute_unknown_vessel_status(df, distance_threshold=0.2):
    """Impute 'unknown' vessel statuses based on the following logic:
    1. vessel_status is 'moored' and sog > 0.2,
    2. vessel_status is 'anchored' and sog > 0.2,
    3. vessel_status is 'moving' and sog == 0,
    4. vessel_status is 'unknown': Impute as follows:
        - If vessel is close to the port (within distance_threshold) and SOG = 0, set vessel_status to 'moored'.
        - If vessel is far from port and latitude/longitude are not consistent between previous/next rows, set vessel_status to 'moving'.
    
    Parameters:
    df (pd.DataFrame): The input dataframe containing vessel_status, sog, latitude, longitude, and port information.
    distance_threshold (float): The distance threshold (in kilometers) to determine if a vessel is close to a port.

    Returns:
    pd.DataFrame: The dataframe with updated vessel_status for the suspicious cases.
    """
    # Set vessel_status to 'unknown' where vessel_status is NaN
    df['vessel_status'] = df['vessel_status'].fillna('unknown')
    
    # Set vessel_status to 'unknown' for moored/anchored with SOG > 0.2 or moving with SOG = 0
    df.loc[
        ((df['vessel_status'] == 'moored') & (df['sog'] > 0.1)) |
        ((df['vessel_status'] == 'anchored') & (df['sog'] > 0.1)) |
        ((df['vessel_status'] == 'moving') & (df['sog'] == 0)),
        'vessel_status'
    ] = 'unknown'
    
    # Vectorized distance calculation using geopy's geodesic function
    def calculate_distance_vectorized(lat1, lon1, lat2, lon2):
        distances = np.zeros(len(lat1))
        for i in range(len(lat1)):
            if not np.isnan(lat2[i]) and not np.isnan(lon2[i]):
                distances[i] = geodesic((lat1[i], lon1[i]), (lat2[i], lon2[i])).km
            else:
                distances[i] = float('inf')  # If port information is missing, return a large distance
        return distances

    # Calculate distances to ports for rows with 'unknown' vessel_status
    mask_unknown = (df['vessel_status'] == 'unknown')
    distances_to_port = calculate_distance_vectorized(
        df.loc[mask_unknown, 'latitude'].values,
        df.loc[mask_unknown, 'longitude'].values,
        df.loc[mask_unknown, 'portLatitude'].values,
        df.loc[mask_unknown, 'portLongitude'].values
    )
    
    # Assign the calculated distances only to the rows with 'unknown' status
    df.loc[mask_unknown, 'distance_to_port'] = distances_to_port
    
    # Set 'unknown' status to 'moored' if the vessel is close to a port and SOG = 0
    df.loc[
        (df['vessel_status'] == 'unknown') & (df['distance_to_port'] <= distance_threshold) & (df['sog'] == 0),
        'vessel_status'
    ] = 'moored'

    # Vectorized check for latitude/longitude consistency to determine movement
    lat_diff = df['latitude'].diff().abs().fillna(0)
    lon_diff = df['longitude'].diff().abs().fillna(0)
    is_moving_prev_next = (lat_diff > 0.0001) | (lon_diff > 0.0001)
    
    # Forward-fill and backward-fill consistency to account for movement in previous and next rows
    is_moving = is_moving_prev_next | is_moving_prev_next.shift(-1).fillna(False) | is_moving_prev_next.shift(1).fillna(False)
    
    # Set 'unknown' to 'moving' if latitude/longitude inconsistency detected
    df.loc[
        (df['vessel_status'] == 'unknown') & is_moving,
        'vessel_status'
    ] = 'moving'
    
    # Clean up intermediate distance column
    df = df.drop(columns=['distance_to_port'])
    
    return df

# Apply the function to your dataset
baseDataset = impute_unknown_vessel_status(baseDataset)


# We now find outliers for sog and impute new values on them

In [20]:
def handle_sog_outliers_with_vessel_status(df, sog_threshold=30):
    """
    Handle SOG outliers based on time_diff and vessel status.
    """
    sog_col = 'sog'
    vessel_col = 'vesselId'
    time_diff_col = 'time_diff'
    vessel_status_col = 'vessel_status'

    # Calculate the median time_diff and SOG for moving vessels for each vessel
    vessel_median_time_diff = df.groupby(vessel_col)[time_diff_col].transform('median')
    vessel_median_moving_sog = df[df[vessel_status_col] == 'moving'].groupby(vessel_col)[sog_col].transform('median')

    # Flag SOG outliers
    df['sog_outlier'] = (df[sog_col] > sog_threshold).astype(int)

    # Handle records based on vessel status
    df.loc[(df['sog_outlier'] == 1) & (df[vessel_status_col] != 'moving'), sog_col] = 0

    # Handle moving vessels by interpolating SOG where necessary
    df.loc[(df['sog_outlier'] == 1) & (df[vessel_status_col] == 'moving') & (df[time_diff_col] <= vessel_median_time_diff), sog_col] = np.nan
    
    # Apply interpolation, ensuring vesselId doesn't become the index
    df[sog_col] = df.groupby(vessel_col)[sog_col].apply(lambda group: group.interpolate(method='linear')).reset_index(level=0, drop=True)

    # Impute SOG based on next time step's vessel status
    def impute_sog(group):
        for idx in group.index:
            if pd.isnull(group.loc[idx, sog_col]) and group.loc[idx, time_diff_col] > vessel_median_time_diff.loc[idx]:
                next_idx = group.index.get_loc(idx) + 1
                if next_idx < len(group):
                    next_status = group.iloc[next_idx][vessel_status_col]
                    if next_status == 'moored':
                        group.loc[idx, sog_col] = vessel_median_moving_sog.loc[idx]
                    else:
                        group.loc[idx, sog_col] = group[sog_col].bfill().iloc[0]
        return group

    # Apply imputation across vessels
    df = df.groupby(vessel_col).apply(impute_sog).reset_index(drop=True)

    # Drop intermediate column
    df = df.drop(columns=['sog_outlier'])

    return df

# Apply the function to handle SOG outliers considering vessel status, time_diff, and gap flags
baseDataset = handle_sog_outliers_with_vessel_status(baseDataset, sog_threshold=35)


There are also outliers where sog is set to 0 which it obviously shouldn't be! So we fix this now

In [21]:
def impute_sog_for_moving(df):
    """
    Impute SOG values for 'moving' vessels with SOG = 0 by:
    1. Checking future/previous SOG values for interpolation or filling.
    2. Estimating speed based on distance between latitude/longitude when both SOG values are 0.

    Parameters:
    df (pd.DataFrame): The input dataframe containing SOG, vessel_status, latitude, longitude, and time_diff columns.

    Returns:
    pd.DataFrame: The dataframe with imputed SOG values for 'moving' vessels with SOG = 0.
    """
    sog_col = 'sog'
    vessel_status_col = 'vessel_status'
    time_diff_col = 'time_diff'
    vessel_col = 'vesselId'
    lat_col = 'latitude'
    lon_col = 'longitude'
    
    # Mask to select rows where vessel is moving and SOG = 0
    mask_moving_sog_zero = (df[vessel_status_col] == 'moving') & (df[sog_col] == 0)
    
    # Step 1: Interpolate or fill SOG values where possible
    def fill_sog_with_neighbors(group):
        # Interpolate where possible (linear)
        group[sog_col] = group[sog_col].replace(0, np.nan)
        group[sog_col] = group[sog_col].interpolate(method='linear', limit_direction='both')
        # Forward/Backward fill if interpolation can't be applied
        group[sog_col] = group[sog_col].fillna(method='ffill').fillna(method='bfill')
        return group
    
    # Apply the function to fill missing SOG values (this works for when there's valid data nearby)
    df.loc[mask_moving_sog_zero, sog_col] = df.groupby(vessel_col, group_keys=False).apply(fill_sog_with_neighbors)[sog_col]
    
    # Step 2: For remaining cases, estimate speed based on latitude/longitude and time_diff
    remaining_zero_sog_mask = (df[sog_col] == 0) & (df[vessel_status_col] == 'moving')
    
    def calculate_speed_from_distance(row):
        # Calculate distance from previous row (latitude/longitude)
        prev_lat, prev_lon = row[lat_col].shift(1), row[lon_col].shift(1)
        curr_lat, curr_lon = row[lat_col], row[lon_col]
        dist = geodesic((prev_lat, prev_lon), (curr_lat, curr_lon)).km
        # Estimate speed: distance / time_diff
        speed = dist / (row[time_diff_col] / 60)  # time_diff in minutes
        return speed

    # Apply speed calculation only where SOG = 0 and vessel is moving
    df.loc[remaining_zero_sog_mask, sog_col] = df.loc[remaining_zero_sog_mask].apply(calculate_speed_from_distance, axis=1)

    return df

baseDataset = impute_sog_for_moving(baseDataset)

# We now fix cog values

In [22]:
def handle_cog_nan_with_interpolation(df):
    """
    Impute COG values for moving vessels using interpolation, and handle moored vessels separately.
    """
    cog_col = 'cog'               # COG column
    vessel_status_col = 'vessel_status'  # Vessel status column (moving, moored, anchored)
    vessel_col = 'vesselId'        # Vessel identifier column

    # Ensure 'vesselId' is not an index
    if vessel_col in df.index.names:
        df = df.reset_index(drop=False)  # Use drop=False to keep existing columns intact

    # Step 1: Set COG to 0 for moored vessels (stationary), as course is not meaningful
    df.loc[df[vessel_status_col] == 'moored', cog_col] = 0
    
    # Step 2: Interpolate COG values for moving vessels
    df[cog_col] = df.groupby(vessel_col)[cog_col].apply(lambda group: group.interpolate(method='linear')).reset_index(level=0, drop=True)

    # Step 3: Forward fill any remaining NaN values (in case interpolation can't be applied, e.g., at start of series)
    df[cog_col] = df.groupby(vessel_col)[cog_col].fillna(method='ffill')

    # Step 4: Backward fill any remaining NaN values (in case interpolation can't be applied, e.g., at end of series)
    df[cog_col] = df.groupby(vessel_col)[cog_col].fillna(method='bfill')

    # Step 5: Ensure COG values are within the valid range [0, 360]
    df[cog_col] = df[cog_col] % 360

    return df

# Apply the function to your dataset
baseDataset = handle_cog_nan_with_interpolation(baseDataset)



In [23]:
baseDataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522033 entries, 0 to 1522032
Data columns (total 24 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   time            1522033 non-null  datetime64[ns]
 1   cog             1522033 non-null  float64       
 2   sog             1522033 non-null  float64       
 3   rot             1522033 non-null  float64       
 4   heading         1517137 non-null  float64       
 5   etaRaw          766797 non-null   datetime64[ns]
 6   latitude        1522033 non-null  float64       
 7   longitude       1522033 non-null  float64       
 8   vesselId        1522033 non-null  object        
 9   portId          1520418 non-null  object        
 10  portLatitude    1520418 non-null  float64       
 11  portLongitude   1520418 non-null  float64       
 12  shippingLineId  1522033 non-null  object        
 13  CEU             1522033 non-null  int64         
 14  DWT             15

# We now fix heading values

In [26]:
from math import atan2, radians, degrees, sin, cos

def calculate_initial_compass_bearing(lat1, lon1, lat2, lon2):
    """
    Calculate the bearing between two points (lat1, lon1) and (lat2, lon2).
    Returns the bearing in degrees (0° is north).
    """
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1

    x = sin(dlon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)

    initial_bearing = atan2(x, y)
    initial_bearing = degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

def handle_heading_with_vessel_status(df):
    """
    Handle heading missing values and anomalies based on time_diff, vessel status, and latitude/longitude.
    """
    heading_col = 'heading'
    vessel_col = 'vesselId'
    time_diff_col = 'time_diff'
    vessel_status_col = 'vessel_status'
    lat_col = 'latitude'
    lon_col = 'longitude'

    # Step 1: Set heading to 0 for moored vessels
    df.loc[df[vessel_status_col] == 'moored', heading_col] = 0

    # Step 2: Calculate heading for rows with NaN based on consecutive lat/lon values
    def calculate_heading_for_vessel(group):
        for idx in group.index:
            if pd.isnull(group.loc[idx, heading_col]):
                next_idx = group.index.get_loc(idx) + 1
                if next_idx < len(group):
                    # Get current and next latitude/longitude
                    curr_lat, curr_lon = group.loc[idx, lat_col], group.loc[idx, lon_col]
                    next_lat, next_lon = group.iloc[next_idx][lat_col], group.iloc[next_idx][lon_col]
                    
                    # Calculate heading between current and next position
                    if not (pd.isnull(curr_lat) or pd.isnull(curr_lon) or pd.isnull(next_lat) or pd.isnull(next_lon)):
                        calculated_heading = calculate_initial_compass_bearing(curr_lat, curr_lon, next_lat, next_lon)
                        group.loc[idx, heading_col] = calculated_heading
                        
        return group

    # Apply the heading calculation function for vessels with missing heading values
    df = df.groupby(vessel_col).apply(calculate_heading_for_vessel).reset_index(drop=True)

    # Step 3: Use transform to apply forward and backward filling while preserving the index
    df[heading_col] = df.groupby(vessel_col)[heading_col].transform(lambda group: group.ffill().bfill())

    return df



baseDataset = handle_heading_with_vessel_status(baseDataset)

In [27]:
baseDataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522033 entries, 0 to 1522032
Data columns (total 24 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   time            1522033 non-null  datetime64[ns]
 1   cog             1522033 non-null  float64       
 2   sog             1522033 non-null  float64       
 3   rot             1522033 non-null  float64       
 4   heading         1522033 non-null  float64       
 5   etaRaw          766797 non-null   datetime64[ns]
 6   latitude        1522033 non-null  float64       
 7   longitude       1522033 non-null  float64       
 8   vesselId        1522033 non-null  object        
 9   portId          1520418 non-null  object        
 10  portLatitude    1520418 non-null  float64       
 11  portLongitude   1520418 non-null  float64       
 12  shippingLineId  1522033 non-null  object        
 13  CEU             1522033 non-null  int64         
 14  DWT             15

## We now fix rot

In [30]:
def handle_rot_for_moored_vessels(df):
    """
    Set ROT (Rate of Turn) to 0 for all moored vessels.
    
    Parameters:
    df (pd.DataFrame): The input dataframe containing vessel_behaviour and rot columns.
    rot_col (str): The name of the ROT column.
    behaviour_col (str): The name of the vessel behaviour column (e.g., 'moored', 'moving').
    
    Returns:
    pd.DataFrame: The dataframe with ROT set to 0 for moored vessels.
    """
    # Set ROT to 0 where vessel_behaviour is 'moored'
    df.loc[df['vessel_status'] == 'moored', 'rot'] = 0
    
    return df

# Apply the function to the dataframe
baseDataset = handle_rot_for_moored_vessels(baseDataset)


# We have now fixed all major issues. However, for our dataset to work on all types of ML models we will give the remaining missing values a value similar to "unknown"

In [32]:
def handle_missing_values(df):
    # For object (categorical or string) features, use 'unknown'
    df['portId'] = df['portId'].fillna('unknown')
    
    # For datetime columns, use a placeholder date like '1970-01-01'
    df['etaRaw'] = df['etaRaw'].fillna(pd.Timestamp('1970-01-01'))
    
    # For numeric columns, use an out-of-bounds value (e.g., -999)
    df['portLatitude'] = df['portLatitude'].fillna(-999)
    df['portLongitude'] = df['portLongitude'].fillna(-999)
    
    
    return df

# Apply the function to handle missing values in your dataset
baseDataset = handle_missing_values(baseDataset)

In [33]:
baseDataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522033 entries, 0 to 1522032
Data columns (total 24 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   time            1522033 non-null  datetime64[ns]
 1   cog             1522033 non-null  float64       
 2   sog             1522033 non-null  float64       
 3   rot             1522033 non-null  float64       
 4   heading         1522033 non-null  float64       
 5   etaRaw          1522033 non-null  datetime64[ns]
 6   latitude        1522033 non-null  float64       
 7   longitude       1522033 non-null  float64       
 8   vesselId        1522033 non-null  object        
 9   portId          1522033 non-null  object        
 10  portLatitude    1522033 non-null  float64       
 11  portLongitude   1522033 non-null  float64       
 12  shippingLineId  1522033 non-null  object        
 13  CEU             1522033 non-null  int64         
 14  DWT             15

# We now move onto feature engineering 